## 訓練・開発データを利用した学習
訓練時に使っていないデータで評価することで過学習を抑える <br />
 - これまで通り訓練データに対する損失を小さくするように学習する <br />
 - エポックごとに未知のデータ（開発データ）に適用するとどの程度の損失になるか確認する

### データ・モデルの読み込み

 - データは極性判定コーパスの一部を利用 <br />
 - モデルは多層パーセプトロン（MLP）を利用 <br />
 - 文章をBoWで表現し，MLPで分類

In [10]:
import random

from tqdm import tqdm
import MeCab
import torch
from torch import optim

from data import Sentence
from data import DataSet
from data import DataLoader
from mlp import MultiLayer # 多層パーセプトロンを利用

mecab_parser = MeCab.Tagger("-Owakati -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd")
mecab_parser.parse("")

'\n'

In [11]:
def load_document(path):
    """
    データ読み込み用関数
    Args:
        path:対象ファイルのパス
    Return:
        tag_list (List[string])  :一行ごとのタグを要素に持つリスト
        sentences_list (List[string]):一行ごとの文を要素に持つリスト
    """
    with open(path) as f:
        texts = f.readlines()

    sentences_list,tag_list = [],[]
    for row in texts:
        sentence_list = []
        tag,words = map(str,row.split("\t"))
        tag_list.append(tag)
        node = mecab_parser.parseToNode(words.replace('\n',''))
        while node:
            analysis = node.feature.split(',')
            if analysis[0] != 'BOS/EOS':
                if analysis[0] == '名詞' and node.surface != '':
                    sentence_list.append(node.surface)
                elif analysis[6] != '':
                    sentence_list.append(analysis[6])
            node = node.next
        sentences_list.append(sentence_list)
    return tag_list, sentences_list

In [12]:
# train, devデータのパス
train_path = "data/train.txt"
dev_path = "data/dev.txt"

# 文書の読み込み
train_label, train = load_document(train_path)
dev_label, dev = load_document(dev_path)

In [13]:
# 訓練・開発データの読み込み
train_data = DataSet(train, train_label)
train_data_size = len(train_data)
dev_data = DataSet(dev, dev_label, train_data.vocab)
dev_data_size = len(dev_data)

In [18]:
# BoW作成のために語彙辞書を出力
output_file = open("vocab.txt", "w")
for v in train_data.vocab:
    output_file.write(v + "\n")

# ラベル辞書を出力（後で）
output_file = open("label_dict.tsv", "w")
for label_name, label_id in train_data.label_dict.items():
    output_file.write(label_name +"\t"+ str(label_id) + "\n")

In [19]:
# モデル
multi_layer_model = MultiLayer(len(train_data.vocab), len(train_data.label_dict))

# 損失の最小化のための最適化手法
op = optim.SGD(multi_layer_model.parameters(), lr=0.1) # lr:learning rate (学習率)

### 学習と評価
訓練データはこれまで通りミニバッチ学習<br />
エポックごとに開発データで損失を計算して，その時点でのモデルを評価

In [20]:
# ミニバッチサイズ
minibatch_size = 10
max_batch_no = train_data_size // minibatch_size

In [21]:
min_dev_loss = 1000 # 開発データで良い評価（損失小）のモデルを保存したいので損失の最小値を得るための初期値を設定
for epoch in range(10):

    # 学習
    print("epoch: ", epoch)

    # モデルを学習モードに（パラメータの勾配計算や更新が可能）
    multi_layer_model.train()

    # ミニバッチ学習のためのデータローダー
    train_data_loader = DataLoader(train_data, batch_size=minibatch_size, shuffle=True)

    # ミニバッチごとに逆伝搬とパラメータ更新
    for batch_no, (batch_input_vector, batch_target_values) in enumerate(train_data_loader):
        # 順伝搬と損失計算
        loss = multi_layer_model.forward_loss(batch_input_vector, batch_target_values)
        if (batch_no + 1) % 10 == 0:
            print("batch_no: {}/{} loss: {}".format(batch_no+1, max_batch_no, loss))

        # 逆伝搬 (勾配の設定)
        loss.backward()

        # パラメータの更新
        op.step()

        # 勾配の消去
        multi_layer_model.zero_grad()


    # epochごとに評価

    # モデルを評価モードに（パラメータの勾配計算や更新が不可）
    multi_layer_model.eval()

    # trainデータのtotal lossを計算 (基本的に下がっていくはず)
    train_data_loader = DataLoader(train_data, batch_size=minibatch_size, shuffle=False)
    eval_train_loss = multi_layer_model.evaluate(train_data_loader)

    # devデータのtotal lossを計算 (未知データなのでtrainデータよりは大きく，下がりづらいことが多い)
    dev_data_loader = DataLoader(dev_data, batch_size=minibatch_size, shuffle=False)
    eval_dev_loss = multi_layer_model.evaluate(dev_data_loader)

    print("epoch: {}  train_loss: {}  dev_loss: {}".format(epoch, eval_train_loss, eval_dev_loss))       

    # 損失最小の場合モデルを保存
    if eval_dev_loss < min_dev_loss:
        print("Epoch{} model saved".format(epoch))
        torch.save(multi_layer_model.state_dict(), "output_models/best_model")
        min_dev_loss = eval_dev_loss
        
    print("="*10)


epoch:  0
batch_no: 10/80 loss: 0.682607889175415
batch_no: 20/80 loss: 0.6782971620559692
batch_no: 30/80 loss: 0.6770572066307068
batch_no: 40/80 loss: 0.7268162965774536
batch_no: 50/80 loss: 0.6980679631233215
batch_no: 60/80 loss: 0.6825952529907227
batch_no: 70/80 loss: 0.6761987209320068
batch_no: 80/80 loss: 0.6759682893753052
epoch: 0  train_loss: 0.6571345955133439  dev_loss: 0.6751232892274857
Epoch0 model saved
epoch:  1
batch_no: 10/80 loss: 0.6337900161743164
batch_no: 20/80 loss: 0.5982596278190613
batch_no: 30/80 loss: 0.6379952430725098
batch_no: 40/80 loss: 0.607840359210968
batch_no: 50/80 loss: 0.5912231206893921
batch_no: 60/80 loss: 0.5052106380462646
batch_no: 70/80 loss: 0.6442306041717529
batch_no: 80/80 loss: 0.6346947550773621
epoch: 1  train_loss: 0.5647016774863005  dev_loss: 0.6361295118927955
Epoch1 model saved
epoch:  2
batch_no: 10/80 loss: 0.6207594871520996
batch_no: 20/80 loss: 0.7911310195922852
batch_no: 30/80 loss: 0.48791423439979553
batch_no: 40